In [1]:
import pandas as pd
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
import time

In [153]:
path = "C:\\Program Files (x86)\\chromedriver.exe" #Download the chrome driver that suits you
service = Service(path)
driver = webdriver.Chrome(service=service)
#Call the method that waits a maximum of 10 seconds to avoid being affected by internet slowness.
driver.implicitly_wait(10)
url = "https://www.emlakjet.com/satilik-konut/istanbul/?kullanim_durumu[]=1"
driver.get(url) #Open the above page from Chrome

In [4]:
#run this cell just for the first time. Because you may lose data

keys = [
    "Net Square Meter",          # Net Metrekare
    "Number of Rooms",           # Oda Sayısı
    "Floor Number",              # Bulunduğu Kat
    "Heating Type",              # Isınma Tipi
    "Credit Eligibility",        # Krediye Uygunluk
    "Number of Bathrooms",       # Banyo Sayısı
    "Number of WC",               #WC Sayısı
    "Gross Square Meter",        # Brüt Metrekare
    "Building Age",              # Binanın Yaşı
    "Number of Floors in Building", # Binanın Kat Sayısı
    "Within Site",               # Site İçerisinde
    "Balcony",                    #Balkon Durumu
    "house is furnished",         #Eşya Durumu
    "Price",                     #We will add this column
    "Location"                   #We will add this column
]

df = pd.DataFrame(columns=keys)

In [5]:
#This function allows me to scrape only the features I want from the page.
def add_values_to_df(df, values):
    labels = ['Net Metrekare', 'Oda Sayısı', 'Bulunduğu Kat',
             'Isıtma Tipi', 'Krediye Uygunluk', 'Banyo Sayısı', 'WC Sayısı', 'Brüt Metrekare',
             'Binanın Yaşı', 'Binanın Kat Sayısı', 'Site İçerisinde', 'Balkon Durumu','Eşya Durumu', 'Price', 'Location']
    extracted = extract_values(values,labels)
    print(extracted)
    df.loc[len(df)] = extracted

    return df

In [6]:
#Add data to dataframe with this function.
def extract_values(values, labels):
    extracted_values = []
    
    for label in labels:
        if label in values:
            index = values.index(label)
            if index + 1 < len(values):
                 extracted_values.append(values[index + 1])
            else:
                extracted_values.append(None)  
        else:
            extracted_values.append(None)  
            
    return extracted_values

In [16]:
try:
    filter_button = WebDriverWait(driver, 10).until(
        EC.visibility_of_element_located((
            By.XPATH,'//*[@id="__next"]/div[3]/div[1]/div/div[5]/div/div/div[3]/div/button')))
    filter_button.click()
    checkboxes = WebDriverWait(driver, 10).until(
        EC.presence_of_all_elements_located((By.XPATH, '//*[@id[contains(., "district-")]]/label'))
    )
    WebDriverWait(driver, 10).until( EC.element_to_be_clickable(filter_button)).click()
    
    for i in range(0,len(checkboxes)): #Creates an for loop for districts
        print(f"{i}. District")
        try:
            #Open the filter tab, select the district and then apply the filters.
            time.sleep(2)
            filter_button = WebDriverWait(driver, 10).until(
                EC.visibility_of_element_located((By.XPATH,'//*[@id="__next"]/div[3]/div[1]/div/div[5]/div/div/div[3]/div/button')))
            filter_button.click()
            
            checkboxes = WebDriverWait(driver, 10).until(
                EC.presence_of_all_elements_located((By.XPATH, '//*[@id[contains(., "district-")]]/label')))
            WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable(checkboxes[i])).click()

            use_filter_button = WebDriverWait(driver, 10).until(
                EC.visibility_of_element_located((
                    By.XPATH,'/html/body/div[4]/div/ul/div[2]/div/div/button')))
            use_filter_button.click()
            time.sleep(2)
            myclass= "_3qUI9q"
            #Identify all adverts on the page that opens
            try:
                adverts = WebDriverWait(driver, 10).until(
                    EC.presence_of_all_elements_located((By.XPATH, '//*[@id="listing-search-wrapper"]/div'))
                )
                while True:
                    adverts = WebDriverWait(driver, 10).until(
                    EC.presence_of_all_elements_located((By.XPATH, '//*[@id="listing-search-wrapper"]/div')))
                    for advert in adverts:
                        try:
                            #get price, location and other features from the page
                            if myclass in advert.get_attribute('class'):
                                price = advert.find_element(By.XPATH, './/a/div[3]/div/div[3]/div/p/span/span').text
                                location = advert.find_element(By.XPATH, './/a/div[3]/div/div[4]/span').text
                                
                                WebDriverWait(driver, 15).until( EC.element_to_be_clickable(advert)).click()

                                #If the button will be pressed, use visibility_of_element_located, if not, use presence_of_all_elements_located
                                more_button = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.CSS_SELECTOR, '.Gp4epw')))
                                more_button.click()
                                values = ''
                                container = WebDriverWait(driver,5).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, '._3tH_Nw._3__dNM')))
                                for j in container:
                                    values = values + j.text
                                    
                                values = values.split("\n")
                                all_values = values[::]
                                all_values.append("Price")
                                all_values.append(price)
                                all_values.append("Location")
                                all_values.append(location)
                
                                df = add_values_to_df(df,all_values)
             
                                driver.execute_script("window.history.go(-1)")
            
                        except StaleElementReferenceException:
                            adverts = WebDriverWait(driver, 10).until(
                                EC.presence_of_all_elements_located((By.XPATH, '//*[@id="listing-search-wrapper"]/div'))
                            )
                            continue 
                    try:
                        #If all adverts are finished, skip to the next page.
                        next_button = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.CSS_SELECTOR,'._3au2n_.OTUgAO')))
                        next_button.click()
                        print("Sonraki sayfaya geçiliyor...")
                        time.sleep(2)
                        
                    except:
                        print("Sonraki sayfa yok veya tüm ilanlar bitti.")
                        break
            
            except Exception as e:
                print("Hata oluştu:ilk", e)
        
            #change filters after scraping data from all pages
            filter_button = WebDriverWait(driver, 15).until(
                EC.visibility_of_element_located((By.XPATH,'//*[@id="__next"]/div[3]/div[1]/div/div[5]/div/div/div[3]/div/button')))
            filter_button.click()
            
            checkboxes = WebDriverWait(driver, 10).until(
                EC.presence_of_all_elements_located((By.XPATH, '//*[@id[contains(., "district-")]]/label')))
            WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable(checkboxes[i])
            ).click()
             
            use_filter_button = WebDriverWait(driver, 10).until(
                EC.visibility_of_element_located((By.XPATH,'/html/body/div[4]/div/ul/div[2]/div/div/button')))
            WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable(use_filter_button)
            ).click()
           
        except StaleElementReferenceException:
            checkboxes = WebDriverWait(driver, 10).until(
                EC.presence_of_all_elements_located((By.XPATH, '//*[@id[contains(., "district-")]]/label')))
            continue 
except Exception as e:
    print(f"Bir hata oluştuson: {e}")


8. District
Sonraki sayfaya geçiliyor...
Sonraki sayfaya geçiliyor...
Sonraki sayfaya geçiliyor...
['80 m²', '2+1', '1.Kat', 'Kombi Doğalgaz', 'Krediye Uygun', '2', None, '90 m²', '0 (Yeni)', '4', 'Hayır', 'Var', 'Boş', '4.700.000', 'İstanbul - Bayrampaşa - Muratpaşa Mh.']
['165 m²', '4+2', '3.Kat', 'Kombi Doğalgaz', 'Krediye Uygun', '4', None, '185 m²', '0 (Yeni)', '4', 'Hayır', 'Var', 'Boş', '7.800.000', 'İstanbul - Bayrampaşa - Muratpaşa Mh.']
['135 m²', '2+2', 'Çatı Dubleks', 'Kombi Doğalgaz', 'Krediye Uygun', '2', '2', '140 m²', '0 (Yeni)', '5', 'Hayır', 'Var', 'Boş', '5.800.000', 'İstanbul - Bayrampaşa - Muratpaşa Mh.']
['86 m²', '2+1', '3.Kat', 'Kombi Doğalgaz', 'Krediye Uygun', '1', None, '90 m²', '16-20', '4', 'Hayır', 'Yok', 'Boş', '2.800.000', 'İstanbul - Bayrampaşa - Vatan Mh.']
['100 m²', '2+1', '1.Kat', 'Kombi Doğalgaz', 'Krediye Uygun', '1', None, '110 m²', '0 (Yeni)', '4', 'Hayır', None, 'Boş', '3.950.000', 'İstanbul - Bayrampaşa - Altıntepsi Mh.']
['55 m²', '1+1', 'Yük

In [177]:
#After you finish scrape data, run this line to load the previous data.
df = pd.read_csv('adverts.csv')

,Net Square Meter,Number of Rooms,Floor Number,Heating Type,Credit Eligibility,Number of Bathrooms,Number of WC,Gross Square Meter,Building Age,Number of Floors in Building,Within Site,Balcony,house is furnished,Price,Location
0,105 m²,4+1,3.Kat,Klimalı,Krediye Uygun,2,NaN,168 m²,21 Ve Üzeri,3,Hayır,NaN,NaN,17.500.000,İstanbul - Adalar - Nizam Mh.
1,300 m²,7+3,NaN,Kombi Doğalgaz,Krediye Uygun,3,NaN,385 m²,16-20,2,Hayır,NaN,NaN,38.000.000,İstanbul - Adalar - Nizam Mh.
2,305 m²,6+3,NaN,Kombi Doğalgaz,Krediye Uygun,5,NaN,380 m²,21 Ve Üzeri,2,Hayır,NaN,Eşyalı,30.500.000,İstanbul - Adalar - Heybeliada Mh.
3,78 m²,2+1,2.Kat,Kombi Doğalgaz,Krediye Uygun,1,NaN,90 m²,0 (Yeni),3,Hayır,NaN,Boş,3.500.000,İstanbul - Arnavutköy - Mustafa Kemal Paşa Mh.
4,70 m²,2+1,2.Kat,Kombi Doğalgaz,Krediye Uygun,1,NaN,78 m²,0 (Yeni),4,Hayır,NaN,Boş,2.650.000,İstanbul - Arnavutköy - Boğazköy İstiklal Mh.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14315,115 m²,3+1,1.Kat,Kombi Doğalgaz,Krediye Uygun,2,None,125 m²,11-15,5,Evet,None,Boş,5.700.000,İstanbul - Bahçelievler - Siyavuşpaşa Mh.
14316,85 m²,2+1,1.Kat,Merkezi (Pay Ölçer),Krediye Uygun,1,None,100 m²,4,8,Evet,None,Boş,3.790.000,İstanbul - Bağcılar - Göztepe Mh.
14317,150 m²,4+1,3.Kat,Kombi Doğalgaz,Krediye Uygun,2,None,180 m²,5-10,3,Hayır,None,Boş,7.490.000,İstanbul - Bayrampaşa - Kocatepe Mh.
14318,130 m²,3.5+1,4.Kat,Kombi Doğalgaz,Krediye Uygun Değil,1,None,135 m²,21 Ve Üzeri,5,Hayır,None,Boş,3.000.000,İstanbul - Bahçelievler - Kocasinan Merkez Mh.


In [178]:
#Convert the captured data to a csv file.
df.to_csv('adverts.csv', index=False)